Import modules

In [6]:
import os
import sys
import numpy as np
import pandas as pd
from pathlib import Path

from one.api import ONE

from utils.ibl_utils import (
    prepare_data,
    bin_spiking_data,
    list_brain_regions,
    select_brain_regions,
    load_video_index,
    load_keypoints,
    load_video,
    resize_video,
    load_keypoints
)

Set Paths information

In [2]:
# ---------------
# PREPROCESS DATA
# ---------------
one = ONE(
    base_url="https://openalyx.internationalbrainlab.org",
    password="international", 
    silent=True,
    cache_dir='../../data/ibl'
)

params = {
    "interval_len": 2, 
    "binsize": 0.02, 
    "single_region": False,
    "align_time": 'stimOn_times', 
    "time_window": (-.5, 1.5), 
    "fr_thresh": 0.5
}

beh_names = [
    "choice", 
    "reward", 
    "block",
    "wheel-speed", 
    "whisker-motion-energy", 
]

N_WORKERS = 4

DYNAMIC_VARS = list(filter(lambda x: x not in ["choice", "reward", "block"], beh_names))

ENCODING_TEST_EIDS_PATH = "../../data/encoding_test_eids.txt"

with open(ENCODING_TEST_EIDS_PATH, "r") as f:
    eids = f.read().splitlines()

ANCHOR_CAMERA = "left"

print(f"Number of encoding sessions: {len(eids)}")
print(eids)

Number of encoding sessions: 10
['d23a44ef-1402-4ed7-97f5-47e9a7a504d9', 'db4df448-e449-4a6f-a0e7-288711e7a75a', '3638d102-e8b6-4230-8742-e548cd87a949', '4b7fbad4-f6de-43b4-9b15-c7c7ef44db4b', '03d9a098-07bf-4765-88b7-85f8d8f620cc', '0841d188-8ef2-4f20-9828-76a94d5343a4', '9b528ad0-4599-4a55-9148-96cc1d93fb24', 'f140a2ec-fd49-4814-994a-fe3476f14e66', '687017d4-c9fc-458f-a7d5-0979fe1a7470', 'd04feec7-d0b7-4f35-af89-0232dd975bf0']


Get encoding sessions

In [4]:
for eid_idx, eid in enumerate(eids):
    print(f"Processing session {eid_idx+1}/{len(eids)}: {eid}")
    neural_dict, behave_dict, meta_data, trials_data, _ = prepare_data(
                one, eid, n_workers=N_WORKERS
    )
    print(neural_dict.keys())
    print(behave_dict.keys())

    regions, beryl_reg = list_brain_regions(neural_dict, **params)
    region_cluster_ids = select_brain_regions(neural_dict, beryl_reg, regions, **params)

    binned_spikes, clusters_used_in_bins, intervals = bin_spiking_data(
        region_cluster_ids,
        neural_dict,
        trials_df=trials_data['trials_df'],
        n_workers=1,
        **params
    )
    print(binned_spikes.shape)
    # Keep responsive neurons
    mean_fr = binned_spikes.sum(1).mean(0) / params["interval_len"]
    keep_unit_idxs = np.argwhere(mean_fr > 1/params["fr_thresh"]).flatten()
    binned_spikes = binned_spikes[..., keep_unit_idxs]
    print(binned_spikes.shape)
    kps = load_keypoints(one, eid, ANCHOR_CAMERA)
    print({k: v.shape for k, v in kps.items()})
    # load video index list
    video_index_list, url = load_video_index(one, eid, ANCHOR_CAMERA, intervals)
    
    print(len(video_index_list), video_index_list[:10], url)

    for trial_idx in range(len(intervals)):
        # trial spike
        spike = binned_spikes[trial_idx]
        trial_video = load_video(
            index=video_index_list[trial_idx],
            url=url
        )
        # print(f'Video shape: {trial_video.shape}')
        # print(f"Spike shape: {spike.shape}")
        # exit()
        _, h, w = trial_video.shape
        h = w = 224
        trial_video = resize_video(
            video=trial_video,
            height=h,
            width=w
        )
        trial_kps ={}
        for key, kp in kps.items():
            trial_kps[key] = kp[video_index_list[trial_idx]]
            # check of there is any nan value in the keypoint
            assert not np.isnan(trial_kps[key]).any(), f'Nan value in keypoint {key}'
        data = {
            'video': trial_video,
            **trial_kps,
        }
    break

Processing session 1/10: d23a44ef-1402-4ed7-97f5-47e9a7a504d9


Merge 2 probes for session eid: d23a44ef-1402-4ed7-97f5-47e9a7a504d9


  0%|          | 0/3 [00:00<?, ?it/s]/u/ywang74/.conda/envs/mv/lib/python3.10/site-packages/one/util.py:428: ALFWarning: Multiple revisions: "", "2025-05-30"
  warnings.warn(f'Multiple revisions: {rev_list}', alferr.ALFWarning)
/u/ywang74/.conda/envs/mv/lib/python3.10/site-packages/one/util.py:428: ALFWarning: Multiple revisions: "", "2025-05-31"
  warnings.warn(f'Multiple revisions: {rev_list}', alferr.ALFWarning)
100%|██████████| 3/3 [00:03<00:00,  1.02s/it]


dict_keys(['spike_times', 'spike_clusters', 'cluster_regions'])
dict_keys(['left-whisker-motion-energy', 'right-whisker-motion-energy', 'wheel-speed'])
Use spikes from brain regions:  ['CA1' 'DCO' 'DG' 'Eth' 'IP' 'LP' 'PARN' 'PO' 'SIM' 'SPVI' 'VISam' 'VPLpc'
 'root' 'y']


100%|██████████| 410/410 [00:39<00:00, 10.44it/s]


(410, 100, 1961)
(410, 100, 1237)


NameError: name 'load_keypoints' is not defined

In [7]:
kps = load_keypoints(one, eid, ANCHOR_CAMERA)
print({k: v.shape for k, v in kps.items()})

/u/ywang74/.conda/envs/mv/lib/python3.10/site-packages/one/util.py:428: ALFWarning: Multiple revisions: "", "2025-06-20"
  warnings.warn(f'Multiple revisions: {rev_list}', alferr.ALFWarning)
(S3) ../../data/ibl/danlab/Subjects/DY_016/2020-09-12/001/alf/_ibl_leftCamera.dlc.pqt: 100%|██████████| 33.0M/33.0M [00:00<00:00, 51.6MB/s]
(S3) ../../data/ibl/danlab/Subjects/DY_016/2020-09-12/001/alf/#2025-06-20#/_ibl_leftCamera.features.pqt: 100%|██████████| 4.08M/4.08M [00:00<00:00, 17.9MB/s]


{'nose_tip_x': (218896,), 'nose_tip_y': (218896,), 'pupil_top_r_x': (218896,), 'pupil_top_r_y': (218896,), 'pupil_right_r_x': (218896,), 'pupil_right_r_y': (218896,), 'pupil_bottom_r_x': (218896,), 'pupil_bottom_r_y': (218896,), 'pupil_left_r_x': (218896,), 'pupil_left_r_y': (218896,), 'paw_l_x': (218896,), 'paw_l_y': (218896,), 'paw_r_x': (218896,), 'paw_r_y': (218896,), 'tube_top_x': (218896,), 'tube_top_y': (218896,), 'tube_bottom_x': (218896,), 'tube_bottom_y': (218896,), 'tongue_end_l_x': (218896,), 'tongue_end_l_y': (218896,), 'tongue_end_r_x': (218896,), 'tongue_end_r_y': (218896,)}


In [10]:
import ibllib.io.video as vidio

urls = vidio.url_from_eid(eid, one=one)
url = urls[ANCHOR_CAMERA]  # URL for the left camera
meta = vidio.get_video_meta(url, one=one)
print('get meta info')

[ WARN:0@2096.241] global cap_ffmpeg_impl.hpp:453 _opencv_ffmpeg_interrupt_callback Stream timeout triggered after 48354.084369 ms
[tls @ 0x5425700] Unknown error


AssertionError: Failed to open video file https://ibl.flatironinstitute.org/public/danlab/Subjects/DY_016/2020-09-12/001/raw_video_data/_iblrig_leftCamera.raw.27640f38-ece8-4b52-bf2c-3a87e861d763.mp4